In [7]:
import itertools
## Regular Expression
import re

## Arrays
import numpy as np

## DataFrames
import pandas as pd

## Visualizations
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import matplotlib.colors as colors
%matplotlib inline

## Modeling
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import learning_curve
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

import contractions
import spacy
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

## Warnings
import warnings
from scipy import stats
warnings.filterwarnings('ignore')

# Reading the data and Preparation

In [3]:
#reading the data
df=pd.read_csv('nlp_reviews_cleaned.csv', delimiter=',')

#creating the classes
df['rating_class'] = df['rating'].apply(lambda x: 0 if x <= 2 else 1)
print(df.rating_class.value_counts())

#train data set reduced due to capacity of computing
df_train = df[0:5000]

#splitting data set into train and test sets
X = df_train['clean_text']
y = df_train['rating_class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=5)

1    12080
0     1192
Name: rating_class, dtype: int64


## Inital Scores with best Vectorizer (CountVectorizer) and best n-gram (unigram) 

In [4]:
# Create the word vector with CountVectorizer

count_vect1 = CountVectorizer(ngram_range=(1,1))
count_vect_train1 = count_vect1.fit_transform(X_train)
count_vect_train1 = count_vect_train1.toarray()
count_vect_test1 = count_vect1.transform(X_test)
count_vect_test1 = count_vect_test1.toarray()

In [28]:
# Get the scores with the best vector (according to step1 and step2)

clf1 = LogisticRegression(random_state=1)
clf2 = SVC(kernel = 'linear') 
clf3 = GaussianNB()
clf4 = KNeighborsClassifier()
clf5 = RandomForestClassifier(random_state=1)
clf6 = GradientBoostingClassifier()

labels = ['Logistic Regression', 'Linear SVM', 'Naive Bayes', 'KNN', 'Random Forest', 'GradientBoosting', ]
for clf, label in zip([clf1, clf2, clf3, clf4, clf5, clf6], labels):
    clf.fit(count_vect_train1, y_train)
    y_pred_clf = clf.predict(count_vect_test1)
    cm = confusion_matrix(y_test, y_pred_clf)
    
    print('\n********** [{}] **********\n'.format(label))
    print('1. Accuarcy: {}\n'.format(metrics.accuracy_score(y_test, y_pred_clf)))
    print('2. The F-1 score of the model {}\n'.format(f1_score(y_test, y_pred_clf, average='weighted')))
    print('3. The recall score of the model {}\n'.format(recall_score(y_test, y_pred_clf, average='weighted')))
    print('4. Classification Report:\n{}\n5. Confusion matrix:\n{}\n\n\n'.format(
        classification_report(y_test, y_pred_clf), cm))


********** [Logistic Regression] **********

1. Accuarcy: 0.9

2. The F-1 score of the model 0.8866628460110704

3. The recall score of the model 0.9

4. Classification Report:
              precision    recall  f1-score   support

           0       0.45      0.25      0.32       118
           1       0.92      0.97      0.95      1132

   micro avg       0.90      0.90      0.90      1250
   macro avg       0.69      0.61      0.63      1250
weighted avg       0.88      0.90      0.89      1250

5. Confusion matrix:
[[  29   89]
 [  36 1096]]




********** [Linear SVM] **********

1. Accuarcy: 0.8928

2. The F-1 score of the model 0.8883063890510475

3. The recall score of the model 0.8928

4. Classification Report:
              precision    recall  f1-score   support

           0       0.42      0.35      0.38       118
           1       0.93      0.95      0.94      1132

   micro avg       0.89      0.89      0.89      1250
   macro avg       0.68      0.65      0.66      12

# 1. Logistic Regression

## 1.1 Logisric Regression - Hyperparameter Tuning

In [4]:
# define the paramater spaces
params = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'penalty': ['l1', 'l2']}
# Instantiate a logistic regression classifier: logreg
logreg = LogisticRegression()
# Instantiate the GridSearchCV object: logreg_cv
logreg_cv = GridSearchCV(logreg, params, cv=5)
# Fit it to the data
logreg_cv.fit(count_vect_train1, y_train)
# Print the tuned parameters and score
print("Tuned Logistic Regression Parameters: {}".format(logreg_cv.best_params_)) 
print("Best score is {}".format(logreg_cv.best_score_))

Tuned Logistic Regression Parameters: {'C': 0.1, 'penalty': 'l2'}
Best score is 0.9178666666666667


## 1.2 Logisric Regression - Modeling with Test Set

In [7]:
label = 'Logistic Regression'
clf = LogisticRegression(C=0.1, penalty='l2')

clf.fit(count_vect_train1, y_train)
y_pred_clf = clf.predict(count_vect_test1)
cm = confusion_matrix(y_test, y_pred_clf)
    
print('\n********** [{}] **********\n'.format(label))
print('1. Accuarcy: {}\n'.format(metrics.accuracy_score(y_test, y_pred_clf)))
print('2. The F-1 score of the model {}\n'.format(f1_score(y_test, y_pred_clf, average='weighted')))
print('3. The recall score of the model {}\n'.format(recall_score(y_test, y_pred_clf, average='weighted')))
print('4. Classification Report:\n{}\n5. Confusion matrix:\n{}\n\n\n'.format(
    classification_report(y_test, y_pred_clf), cm))


********** [Logistic Regression] **********

1. Accuarcy: 0.9064

2. The F-1 score of the model 0.8817491157291112

3. The recall score of the model 0.9064

4. Classification Report:
              precision    recall  f1-score   support

           0       0.52      0.14      0.23       118
           1       0.92      0.99      0.95      1132

   micro avg       0.91      0.91      0.91      1250
   macro avg       0.72      0.56      0.59      1250
weighted avg       0.88      0.91      0.88      1250

5. Confusion matrix:
[[  17  101]
 [  16 1116]]





# 1.3 Logisric Regression - Modeling with Train Set

In [8]:
label = 'Logistic Regression'
clf = LogisticRegression(C=0.1, penalty='l2')

clf.fit(count_vect_train1, y_train)
y_pred_clf = clf.predict(count_vect_train1)
cm = confusion_matrix(y_train, y_pred_clf)
    
print('\n********** [{}] **********\n'.format(label))
print('1. Accuarcy: {}\n'.format(metrics.accuracy_score(y_train, y_pred_clf)))
print('2. The F-1 score of the model {}\n'.format(f1_score(y_train, y_pred_clf, average='weighted')))
print('3. The recall score of the model {}\n'.format(recall_score(y_train, y_pred_clf, average='weighted')))
print('4. Classification Report:\n{}\n5. Confusion matrix:\n{}\n\n\n'.format(
    classification_report(y_train, y_pred_clf), cm))


********** [Logistic Regression] **********

1. Accuarcy: 0.9685333333333334

2. The F-1 score of the model 0.9652290559106603

3. The recall score of the model 0.9685333333333334

4. Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.63      0.77       319
           1       0.97      1.00      0.98      3431

   micro avg       0.97      0.97      0.97      3750
   macro avg       0.98      0.82      0.88      3750
weighted avg       0.97      0.97      0.97      3750

5. Confusion matrix:
[[ 201  118]
 [   0 3431]]





## 1.4 Logisric Regression - Hyperparameter Tuning Result

### Contribution of Hyperparameter tuning (on Test Set): - 0.05 % 

# 2. Linear SVM

## 2.1. Linear SVM - Hyperparameter Tuning

In [10]:
# Specify the hyperparameter space
Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.001, 0.01, 0.1, 1]
param_grid = {'C': Cs, 'gamma' : gammas}
svm=SVC(kernel='linear')
svm_cv = GridSearchCV(svm, param_grid, cv=5)

# Fit to the training set
svm_cv.fit(count_vect_train1, y_train)

# Compute and print metrics
print("Accuracy: {}".format(svm_cv.score(count_vect_test1, y_test)))
print("Tuned Model Parameters: {}".format(svm_cv.best_params_))

Accuracy: 0.9056
Tuned Model Parameters: {'C': 0.001, 'gamma': 0.001}


## 2.2. Linear SVM - Modeling with Test Set

In [6]:
label = 'Logistic Regression'
clf = SVC(kernel='linear', C=0.001, gamma=0.001)

clf.fit(count_vect_train1, y_train)
y_pred_clf = clf.predict(count_vect_test1)
cm = confusion_matrix(y_test, y_pred_clf)
    
print('\n********** [{}] **********\n'.format(label))
print('1. Accuarcy: {}\n'.format(metrics.accuracy_score(y_test, y_pred_clf)))
print('2. The F-1 score of the model {}\n'.format(f1_score(y_test, y_pred_clf, average='weighted')))
print('3. The recall score of the model {}\n'.format(recall_score(y_test, y_pred_clf, average='weighted')))
print('4. Classification Report:\n{}\n5. Confusion matrix:\n{}\n\n\n'.format(
    classification_report(y_test, y_pred_clf), cm))


********** [Logistic Regression] **********

1. Accuarcy: 0.9056

2. The F-1 score of the model 0.8607382031905961

3. The recall score of the model 0.9056

4. Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       118
           1       0.91      1.00      0.95      1132

   micro avg       0.91      0.91      0.91      1250
   macro avg       0.45      0.50      0.48      1250
weighted avg       0.82      0.91      0.86      1250

5. Confusion matrix:
[[   0  118]
 [   0 1132]]





# Computations for other models delayed due to limitations of capacity